In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
#from matplotlib import rcParam
import seaborn as sb
import os
import sys
sys.setrecursionlimit(1000000)
from scipy.stats import spearmanr
import pathlib
import scipy as sp
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
mcc = MouseConnectivityCache()
from scipy.stats import pearsonr
import itertools

In [ ]:
from matplotlib import rcParams
#for_binary_barcode_path = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences/filtered_barcode_matrix_for_binary_junk_test.pkl')
#normalised_barcode_path_junk_test = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences/barcode_matrix_normalised_junk_test.pkl')
normalised_barcode_path= pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq_corrected/Final_processed_sequences/barcode_matrix_normalised.pkl')
lcm_reg_dir = pathlib.PurePath('/nemo/lab/znamenskiyp/home/shared/code/MAPseq_processing/AC_MAPseq/Brain1_FIAA32.6a/LCM_registration')
areas = pd.read_csv(lcm_reg_dir/'3d_areas.csv')
RTtosample = pd.read_csv(lcm_reg_dir/'RTprimer_tosample.csv')
areas = areas.merge(RTtosample, how='inner', on='sample')
areas.sort_values("RT_primer", inplace=True)





In [ ]:
#load barcodes, and drop samples that have been qc'd out
barcodes_across_sample = pd.read_pickle(normalised_barcode_path)

areas =areas[areas['RT_primer'].isin(barcodes_across_sample.columns)]

What is the conditional probability of co-projection between samples?

In [ ]:
barcodes_matrix = pd.read_pickle(normalised_barcode_path)
columns = barcodes_matrix.columns.tolist()
#conditional_prob = pd.DataFrame(columns=['A', 'B', 'conditional_probability'])
i = 0
for col_a, col_b in itertools.combinations(columns, 2):
    prob_df = pd.DataFrame()
    prob_df['a'] = barcodes_matrix[col_a].astype(bool)
    prob_df['b'] = barcodes_matrix[col_b].astype(bool)
    prob_df['matching'] = prob_df.sum(axis=1)
    probability_b_a = len(prob_df[prob_df['matching']==2])/prob_df['a'].sum()
    probability_a_b = len(prob_df[prob_df['matching']==2])/prob_df['b'].sum()
    temp = pd.DataFrame({'A': col_a, 'B': col_b, 'conditional_probability_b_to_a': probability_b_a, 'conditional_probability_a_to_b': probability_a_b}, index=[0])
    if i == 0:
        conditional_prob = temp
    else:
        conditional_prob =pd.concat([conditional_prob, temp])
    i = i+1
    
heatmap_df = np.zeros(shape=((len(columns), (len(columns)))))
heatmap_df_matrix = pd.DataFrame(data= heatmap_df, columns=columns, index=columns).replace(0, np.nan)

for i, r in conditional_prob.iterrows():
    heatmap_df_matrix.loc[r['B'], r['A']] = r['conditional_probability_b_to_a']
    heatmap_df_matrix.loc[r['A'], r['B']] = r['conditional_probability_a_to_b']



rcParams['figure.figsize'] = 15, 15
sb.heatmap(heatmap_df_matrix, xticklabels=True, yticklabels=True, cmap ="viridis",  cbar_kws={'label': 'P B|A', "shrink": 0.4})
plt.title('Conditional Probabilities of Shared Barcodes Across All Samples')
plt.xlabel('Sample A')
plt.ylabel('Sample B')
#ax.set(xlabel="", ylabel="")

In [ ]:
#conditional probabilities of co-projection between cortical areas
cortical_areas =[6, 7, 10, 11, 12, 15, 16, 17, 18, 21, 22, 23, 24, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 59, 60, 61, 62, 63, 64, 65, 70, 80, 81, 82, 83, 84]
cortical_areas_adj = list(filter(lambda x: x in barcodes_matrix.columns, cortical_areas))
i = 0
for col_a, col_b in itertools.combinations(cortical_areas_adj, 2):
    prob_df = pd.DataFrame()
    prob_df['a'] = barcodes_matrix[col_a].astype(bool)
    prob_df['b'] = barcodes_matrix[col_b].astype(bool)
    prob_df['matching'] = prob_df.sum(axis=1)
    probability_b_a = len(prob_df[prob_df['matching']==2])/prob_df['a'].sum()
    probability_a_b = len(prob_df[prob_df['matching']==2])/prob_df['b'].sum()
    temp = pd.DataFrame({'A': col_a, 'B': col_b, 'conditional_probability_b_to_a': probability_b_a, 'conditional_probability_a_to_b': probability_a_b}, index=[0])
    if i == 0:
        conditional_prob = temp
    else:
        conditional_prob =pd.concat([conditional_prob, temp])
    i = i+1
    
heatmap_df = np.zeros(shape=((len(cortical_areas_adj), (len(cortical_areas_adj)))))
heatmap_df_matrix = pd.DataFrame(data= heatmap_df, columns=cortical_areas_adj, index=cortical_areas_adj)
heatmap_df_matrix = heatmap_df_matrix.replace(0, np.nan)

for i, r in conditional_prob.iterrows():
    heatmap_df_matrix.loc[r['B'], r['A']] = r['conditional_probability_b_to_a']
    heatmap_df_matrix.loc[r['A'], r['B']] = r['conditional_probability_a_to_b']



rcParams['figure.figsize'] = 15, 10
sb.heatmap(heatmap_df_matrix, xticklabels=True, yticklabels=True, cmap ="viridis",  cbar_kws={'label': 'P B|A', "shrink": 0.4})
plt.title('Conditional Probabilities of Shared Barcodes Across Cortical Samples')
plt.xlabel('Sample A')
plt.yticks(rotation=0)
plt.ylabel('Sample B')

In [ ]:

columns = barcodes_across_sample.columns.tolist()

correlations = {}
for col_a, col_b in itertools.combinations(columns, 2):
    correlations[f"{col_a}" + '__' + f"{col_b}"] = pearsonr(barcodes_across_sample.loc[:, col_a], barcodes_across_sample.loc[:, col_b])

result = pd.DataFrame.from_dict(correlations, orient='index')
result.columns = ['PCC', 'p-value']

print(result.sort_index())

In [ ]:
correlations

In [ ]:
sb.clustermap(barcodes_across_sample, metric='canberra', standard_scale=0, norm=LogNorm(), cmap="Blues", figsize=(20, 10), yticklabels=False)

In [ ]:
#what about just cortex
#no_source = barcodes_across_sample.drop([36, 64, 37, 52, 50, 42, 51, 65], axis=1)
cortex_only = barcodes_across_sample[cortical_areas_adj]
for column in cortex_only.columns:
    if cortex_only[column].sum() == 0:
        cortex_only.drop([column], axis=1, inplace=True)
cortex_only = cortex_only.loc[~(cortex_only==0).all(axis=1)]
sb.clustermap(cortex_only, metric='canberra', standard_scale=0, norm=LogNorm(), cmap="Blues", figsize=(20, 10))


In [ ]:
all_areas_fractions = pd.DataFrame(index=barcodes_across_sample.columns)
all_areas_fractions['name'] = 'string'
areas_to_look = areas.drop('sample', axis=1).set_index('RT_primer')
for i, row in areas_to_look.iterrows():
    row /= row.sum(0)
    
    if row[row.idxmax()]>0.9999:
        name= str(row.idxmax()) + ' ' + str(row[row.idxmax()].round(2))
    else:
        second_highest_row = row.drop(row.idxmax())
        name= str(row.idxmax()) + ' ' + str(row[row.idxmax()].round(2)) + ' ' + str(second_highest_row.idxmax()) + ' ' + str(row[second_highest_row.idxmax()].round(2))
    all_areas_fractions.loc[i] = name
    

In [ ]:
barcodes_across_sample_renamed = barcodes_across_sample.copy()

barcodes_across_sample_renamed.columns = all_areas_fractions['name'].tolist()
barcodes_across_sample_renamed =barcodes_across_sample_renamed.reset_index(drop=True)

In [ ]:
sb.clustermap(barcodes_across_sample_renamed, metric='canberra', standard_scale=0, norm=LogNorm(), cmap="PuBuGn", figsize=(20, 10), yticklabels=False, cbar_pos=(1.01, 0.5, 0.02, 0.18))

In [ ]:
matrix = barcodes_across_sample_renamed.corr()
plt.figure(figsize=(30,20))
mask = np.zeros_like(matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True




#_ = sb.heatmap(matrix, center=0, annot=True, 
 #               fmt='.2f', square=True, cmap='coolwarm')
sb.clustermap(matrix, cmap='PuBuGn', yticklabels=True, xticklabels=True,  cbar_kws={'label': 'pearson r', "shrink": 0.4}, cbar_pos=(1.01, 0.5, 0.02, 0.18))

In [ ]:

sb.clustermap(matrix, metric='euclidean', cmap="PuBuGn", figsize=(10, 10), xticklabels=True, yticklabels=True, cbar_pos=(1.01, 0.5, 0.02, 0.18))

In [ ]:
non_cortex = ['CP 0.95 fa 0.01', 'AId 0.33 CLA 0.23', 'Contra-SSp-tr 0.87 Contra-SSp-ll 0.11', 'CP 0.92 LA 0.06', 'Contra-SSs 0.62 Contra-AUDd 0.19', 'Contra-VISa 0.52 Contra-SSp-tr 0.38', 'Contra-AUDv 0.43 Contra-or 0.2', 'LP 0.42 SGN 0.23', 'POL 0.35 PoT 0.28', 'POL 0.64 TH 0.21', 'MGv 0.59 MGd 0.29', 'PG 0.84 P 0.08', 'ICe 0.47 MB 0.22', 'ICc 0.51 ICe 0.27', 'ICc 0.4 ICd 0.25', 'SCiw 0.69 SCig 0.21', 'SCsg 0.54 SCop 0.31']
cortex_only_matrix = matrix.drop(non_cortex, axis=0)
cortex_only_matrix = cortex_only_matrix.drop(non_cortex, axis=1)


In [ ]:

sb.clustermap(cortex_only_matrix, metric='canberra', cmap="PuBuGn", figsize=(10, 10), xticklabels=True, yticklabels=True, cbar_pos=(1.01, 0.5, 0.02, 0.18))

In [ ]:
reg_dir = pathlib.Path('/nemo/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/allenccf/allen_ccf_coord')
ROI_path = pathlib.Path('/nemo/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/rois')
ROI_table = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/allenccf/ROI_vol.pkl')
ROI_vol = pd.read_pickle(ROI_table)



In [ ]:
import nrrd
anno_path = reg_dir.parents[1]/'reference/annotation_25.nrrd'
allenanno = nrrd.read(anno_path)
allenanno = np.array(allenanno)
annotation = allenanno[0]

In [ ]:
#re-get registration info on each ROI
#iterate through individual slices, take average difference in in coordinates in z (which is x axes in allen ccf) for last slice (slice s001 for brain 1), take average of previous slices
add_z = pd.DataFrame(columns=['slice', 'amountz'], dtype=int)
#need to change for mega thick last bit of cortex section, so extend ROI through 3slices
for file in os.listdir(reg_dir):
    if file.startswith('allen_ccf_converted_'):
        slice_name = file[20:24]
        slicenum = int(file[21:24])
        if slice_name != 's001' and slice_name != 's012' and slice_name != 's048' and slice_name != 's053':
            slice_before= slicenum-1
            if slice_before >9:
                slicebefore_name = f's0{slice_before}' 
            if slice_before<10:
                slicebefore_name = f's00{slice_before}' 
            [x1a, y1a, z1a, one1] = np.load(reg_dir/file)
            [x2a, y2a, z2a, one2] = np.load(reg_dir/f'allen_ccf_converted_{slicebefore_name}.npy')
            dif = np.average(x2a.flatten()-x1a.flatten())
            add_z= add_z.append({'slice': slice_name, 'amountz': dif},ignore_index=True)
#for slices where the one's before are missing, extend them by the mean of slice z extensions for the others
s001_add =add_z['amountz'].mean()
add_z= add_z.append({'slice': 's001', 'amountz': s001_add},ignore_index=True)
s012_add =add_z['amountz'].mean()
add_z= add_z.append({'slice': 's012', 'amountz': s012_add},ignore_index=True)
s048_add =add_z['amountz'].mean()
add_z= add_z.append({'slice': 's048', 'amountz': s048_add},ignore_index=True)
s053_add =add_z['amountz'].mean()
add_z= add_z.append({'slice': 's053', 'amountz': s053_add},ignore_index=True)

In [ ]:
ROI_vol = pd.DataFrame()
for region in os.listdir(ROI_path):
    if region.startswith("s0"):
        slicename = region[0:4]
        tube = region[5:len(region)].split('TUBE', 1)[1]
        tube =tube[:-4]
        converted_slice = reg_dir/ f'allen_ccf_converted_{slicename}.npy'
        [xa, ya, za, one] = np.load(converted_slice)
        roi_path = ROI_path / f'{region}'
        roi = plt.imread(roi_path)
        allencoord_roiya = roi*ya
        allencoord_roiza = roi*za
        z_axis =roi*xa
        z_axis[z_axis == 0] = np.nan
        most_anterior = np.nanmin(z_axis)
#iterate image by z slices, each additional z, annotate then add to list
        z_to_add = add_z.loc[add_z['slice'] == slicename, 'amountz'].iloc[0]
        most_posterior = np.nanmax(z_axis) - z_to_add
        
        ROI_vol= ROI_vol.append({'slice': slicename, 'tube': tube, 'z_added': z_to_add, 'most_anterior': most_anterior, 'most_posterior': most_posterior},ignore_index=True)





In [ ]:
result = ROI_vol.groupby(['tube']).agg(', '.join).reset_index()
result

In [ ]:
result['most_anterior'] = ""
result['most_posterior'] =""
result['RT_primer'] = ""
for i, row in result.iterrows():
    result.loc[i]['most_anterior'] = ROI_vol[ROI_vol['tube'] ==row['tube']].most_anterior.min(axis=0)
    result.loc[i]['most_posterior'] = ROI_vol[ROI_vol['tube'] ==row['tube']].most_posterior.max(axis=0)
    if pd.isnull(RTtosample[RTtosample['sample']==int(row['tube'])].RT_primer).all() == False:
        result.loc[i]['RT_primer'] = int(RTtosample[RTtosample['sample']==int(row['tube'])].RT_primer.values)
    else:
        result.loc[i]['RT_primer'] = 'NA'

In [ ]:
result =result[result['RT_primer'].isin(barcodes_across_sample.columns)]

In [ ]:
result['fraction_name'] = ""
for i, row in result.iterrows():
    result.loc[i]['fraction_name'] = all_areas_fractions.loc[int(row['RT_primer'])]['name']

In [ ]:
result = result.sort_values(by=['most_anterior'])
result = result.set_index('RT_primer')
anterior_pos_sorted = result.T

In [ ]:
barcodes_across_sample_AP_sorted = barcodes_across_sample[anterior_pos_sorted.columns]

barcodes_across_sample_AP_sorted.columns = result.fraction_name.to_list()

In [ ]:
matrix = barcodes_across_sample_AP_sorted.corr()
plt.figure(figsize=(30,20))
mask = np.zeros_like(matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True




_ = sb.heatmap(matrix, cmap='PuBuGn', cbar_kws={'label': 'pearson r', "shrink": 0.4})


In [ ]:
#look at conditional probability ordered in AP axis
heatmap_df_matrix_AP_sorted = heatmap_df_matrix[anterior_pos_sorted.columns]
heatmap_df_matrix_AP_sorted.columns = result.fraction_name.to_list()

In [ ]:
heatmap_df_matrix_AP_sorted = heatmap_df_matrix_AP_sorted.T
heatmap_df_matrix_AP_sorted = heatmap_df_matrix_AP_sorted[anterior_pos_sorted.columns]
heatmap_df_matrix_AP_sorted.columns = result.fraction_name.to_list()
heatmap_df_matrix_AP_sorted= heatmap_df_matrix_AP_sorted.T

In [ ]:
sb.heatmap(heatmap_df_matrix_AP_sorted, cmap = 'PuBuGn',cbar_kws={'label': 'P B|A', "shrink": 0.4})


In [ ]:
matrix = barcodes_across_sample_renamed.corr()
plt.figure(figsize=(30,20))
mask = np.zeros_like(matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True




_ = sb.heatmap(matrix, center=0, annot=True, 
                fmt='.2f', square=True, cmap='coolwarm')


In [ ]:
areas =areas.drop(['sample'], axis=1)
areas =areas.set_index('RT_primer').sort_index()
areas_matrix = areas.to_numpy()
total_frac = np.sum(areas_matrix, axis=1)
frac_matrix = areas_matrix/total_frac[:, np.newaxis]
areasFrac = pd.DataFrame(frac_matrix, columns=areas.columns, index= areas.index)


In [ ]:
areas_matrix = areas.to_numpy()
total_frac = np.sum(areas_matrix, axis=1)
frac_matrix = areas_matrix/total_frac[:, np.newaxis]
areasFrac = pd.DataFrame(frac_matrix, columns=areas.columns, index= areas.index)


In [ ]:
areasFrac[areasFrac['AUDp']>0.1]

In [ ]:
for column in areasFrac.columns:
    print(column, areasFrac.loc[36][column])

Manual annotation of subcortical structures

In [ ]:
regions_to_add = {}
regions_to_add['MG'] = [55, 66]
regions_to_add['LP'] = [30, 39, 54]
regions_to_add['POL'] = [44, 56, 67]
regions_to_add['Rstr'] = [8, 13, 19]
regions_to_add['CLA/Epc'] = [9, 14, 20, 25]
regions_to_add['Cstr'] = [29]
regions_to_add['SCs'] = [87, 68, 78, 92]
regions_to_add['SCm'] = [69, 79, 93]
regions_to_add['IC'] = [86, 89, 90]
regions_to_add['pons'] = [72, 80, 88]


In [ ]:
for k, v in regions_to_add.items():
    if k not in areas.columns:
        areas[k] = 0
    for value in v:
        area_of_interest = areas.loc[areas['sample'] == value]
        new_val = area_of_interest.sum(axis=1)-(area_of_interest['RT_primer']+area_of_interest['sample'])
        to_add = pd.DataFrame(columns=areas.columns)
        to_add[k] = new_val
        to_add['sample']= value
        to_add['RT_primer'] = area_of_interest['RT_primer']
        to_add =to_add.fillna(0)
        areas = areas.drop(area_of_interest.index)
        areas= pd.concat([areas, to_add])
areas =areas.set_index('RT_primer').sort_index()

Grouping LCM sections according to broad areas

In [ ]:
group_areas = {
    'tectum': ['SCs', 'SCm', 'IC'],
    #'tectum': ['SCdg', 'SCdw', 'SCig', 'SCiw', 'SCop', 'SCsg', 'SCzo', 'ICc', 'ICd', 'ICe', 'NB'],
    'thalamus': ['PoT', 'TH', 'MGm', 'MGv', 'MGd', 'LGd-co', 'LP', 'POL', 'PO', 'LD', 'VPL', 'PIL', 'Eth'],
    'SS': ['SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-n', 'SSp-tr', 'SSp-ul', 'SSp-un', 'SSs'],
    'M': ['MOs', 'MOp'],
    'contra': areas.filter(like="Contra").columns,
    #'AudC': ['Contra-AUDd', 'Contra-AUDp', 'Contra-AUDv'],
    #'VisC': ['Contra-VISa', 'Contra-VISam'],
    'VisIP': ['VISa', 'VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISpor', 'VISrl', 'VISli', 'VISpl'],
    #'Rstr': ['CP', 'STR', 'ACB'],
    #'pons': ['SOCm', 'SOCl', 'POR', 'PRNr', 'PRNc', 'TRN', 'P', 'P-mot', 'PG', 'NLL']
}

In [ ]:
areas_grouped = areas.copy()
for group, columns in group_areas.items():
    areas_grouped[group] = areas_grouped.filter(items=columns).sum(axis=1)
    areas_grouped = areas_grouped.drop(columns, axis=1)
areas_only_grouped = areas_grouped.drop(['sample', 'ar', 'bic', 'bsc', 'ccb', 'ccb', 'ccg', 'cing', 'cpd', 'csc', 'cst', 'ec', 'fa', 'fi',
    'fiber tracts', 'fp', 'll', 'mcp', 'ml', 'onl', 'or', 'py', 'root', 'sctv', 'scwm', 'tb', 'CTXsp', 'act', 'alv', 'amc', 'cic', 'VL', 'MRN'], axis=1)

In [ ]:
#areas_comp = ['Cstr', 'tectum', 'thalamus', 'contra', 'pons']
areas_comp = ['M', 'Cstr', 'Rstr', 'tectum', 'thalamus', 'contra', 'VisIP', 'SS']
areas_only_grouped = areas_only_grouped[areas_comp]

In [ ]:
#since we're directly comparing co-projection between different areas, we will drop any assigned areas where two areas
# are in the same sample (keeping only area as one that has most contained)
for i, row in areas_only_grouped.iterrows():
    if row.max()<row.sum():
        max_val_col = areas_only_grouped.loc[i, (row == row.max())].index.tolist()[0]
        max_val = row.max()
        areas_only_grouped.loc[i] = 0
        areas_only_grouped.loc[i][max_val_col] =max_val



In [ ]:
#create a dataframe of the fractions of each brain area contained within each sample
areas_matrix = areas_only_grouped.to_numpy()
total_frac = np.sum(areas_matrix, axis=1)
frac_matrix = areas_matrix/total_frac[:, np.newaxis]
areasFrac = pd.DataFrame(frac_matrix, columns=areas_only_grouped.columns)


In [ ]:
#for each barcode, create a matrix of BC count for regions in a sample based on amount of each region in LCM (makes assumption of equal BC distribution)
bc_matrix = np.zeros(shape=((len(barcodes_across_sample), (len(areas_only_grouped.columns)))))
bc_matrix = pd.DataFrame(data= bc_matrix, columns=areas_only_grouped.columns, index=barcodes_across_sample.index)
for i, row in barcodes_across_sample.iterrows():
    counts = row.to_numpy()
    frac_counts = frac_matrix * counts[:, np.newaxis]
    sample_counts = pd.DataFrame(frac_counts, columns=areas_only_grouped.columns)
    bc_matrix.loc[i] = sample_counts.sum() / areas_only_grouped.sum()
#remove any columns that are all zeros
for column in bc_matrix.columns:
    if bc_matrix[column].sum() == 0:
        bc_matrix.drop([column], axis=1, inplace=True)
bc_matrix = bc_matrix.loc[~(bc_matrix==0).all(axis=1)]



In [ ]:
pt_vs_it = ['Cstr', 'tectum', 'thalamus', 'contra']
striatum_proj = bc_matrix[bc_matrix['Cstr']>0]
striatum_proj = striatum_proj[pt_vs_it]

In [ ]:
striatum_proj =striatum_proj.reset_index(drop=True)
bc_matrix = bc_matrix.reset_index(drop=True)

In [ ]:
# perform hierarchial clustering of all barcodes across samples
#sb.clustermap(bc_matrix, metric='euclidean', standard_scale=0, norm=LogNorm(), cmap="Blues", figsize=(10, 10))

sb.clustermap(striatum_proj, metric='canberra', method='average', standard_scale=0, norm=LogNorm(), cmap="PuBuGn", figsize=(3, 5), yticklabels=False, cbar_pos=(1.01, 0.5, 0.02, 0.18))


In [ ]:
sb.clustermap(bc_matrix, metric='canberra', method='average', standard_scale=0, norm=LogNorm(), cmap="PuBuGn", figsize=(3, 5), yticklabels=False, cbar_pos=(1.01, 0.5, 0.02, 0.18))

In [ ]:
#upper = barcodes_across_sample[barcodes_across_sample.idxmax(axis=1).isin([49, 51, 40, 42])]
#binary_barcodes = pd.read_pickle(no_barcode_path)
upper_index = barcodes_across_sample[barcodes_across_sample.idxmax(axis=1).isin([49, 51, 40, 42])]

for i, row in upper_index.iterrows():
    col= row.idxmax()
    if row[col]/row.drop([col], axis=0).max()<20:
        upper_index = upper_index.drop(i)
        
upper = barcodes_across_sample[barcodes_across_sample.index.isin(upper_index.index)]

In [ ]:

#for each barcode, create a matrix of BC count for regions in a sample based on amount of each region in LCM (makes assumption of equal BC distribution)
bc_matrix = np.zeros(shape=((len(upper), (len(areas_only_grouped.columns)))))
bc_matrix = pd.DataFrame(data= bc_matrix, columns=areas_only_grouped.columns, index=upper.index)
for i, row in upper.iterrows():
    counts = row.to_numpy()
    frac_counts = frac_matrix * counts[:, np.newaxis]
    sample_counts = pd.DataFrame(frac_counts, columns=areas_only_grouped.columns)
    bc_matrix.loc[i] = sample_counts.sum() / areas_only_grouped.sum()
#remove any columns that are all zeros
for column in bc_matrix.columns:
    if bc_matrix[column].sum() == 0:
        bc_matrix.drop([column], axis=1, inplace=True)
bc_matrix = bc_matrix.loc[~(bc_matrix==0).all(axis=1)]
# perform hierarchial clustering of all barcodes across samples
sb.clustermap(bc_matrix, metric='euclidean', standard_scale=0, norm=LogNorm(), cmap="Blues", figsize=(10, 10))

In [ ]:
#binary_barcodes = pd.read_pickle(for_binary_barcode_path)
lower_index = barcodes_across_sample[barcodes_across_sample.idxmax(axis=1).isin([50, 52, 41, 43])]
for i, row in lower_index.iterrows():
    col= row.idxmax()
    if row[col]/row.drop([col], axis=0).max()<20:
        lower_index = lower_index.drop(i)

lower = barcodes_across_sample[barcodes_across_sample.index.isin(lower_index.index)]



#for each barcode, create a matrix of BC count for regions in a sample based on amount of each region in LCM (makes assumption of equal BC distribution)
bc_matrix = np.zeros(shape=((len(lower), (len(areas_only_grouped.columns)))))
bc_matrix = pd.DataFrame(data= bc_matrix, columns=areas_only_grouped.columns, index=lower.index)
for i, row in lower.iterrows():
    counts = row.to_numpy()
    frac_counts = frac_matrix * counts[:, np.newaxis]
    sample_counts = pd.DataFrame(frac_counts, columns=areas_only_grouped.columns)
    bc_matrix.loc[i] = sample_counts.sum() / areas_only_grouped.sum()
#remove any columns that are all zeros
for column in bc_matrix.columns:
    if bc_matrix[column].sum() == 0:
        bc_matrix.drop([column], axis=1, inplace=True)
bc_matrix = bc_matrix.loc[~(bc_matrix==0).all(axis=1)]
# perform hierarchial clustering of all barcodes across samples
sb.clustermap(bc_matrix, metric='euclidean', standard_scale=0, norm=LogNorm(), cmap="Blues", figsize=(10, 10))

In [ ]:
len(bc_matrix[bc_matrix['Cstr']>0])/len(bc_matrix)

Look at projection densities across brain in more detail

In [ ]:
#group the structures that have too much detail
group_areas = {
    #'SC': ['SCdg', 'SCdw', 'SCig', 'SCiw', 'SCop', 'SCsg', 'SCzo'],
    #'IC': ['ICc', 'ICd', 'ICe', 'NB'],
    'SSp': ['SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-n', 'SSp-tr', 'SSp-ul', 'SSp-un'],
    'contra': areas.filter(like="Contra").columns,
    #'AudC': ['Contra-AUDd', 'Contra-AUDp', 'Contra-AUDv'],
    #'VisC': ['Contra-VISa', 'Contra-VISam'],
    #'VisIP': ['VISa', 'VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISpor', 'VISrl', 'VISli', 'VISpl'],
    #'pons': ['SOCm', 'SOCl', 'POR', 'PRNr', 'PRNc', 'TRN', 'P', 'P-mot', 'PG', 'NLL']
}

In [ ]:
#load barcodes, and drop samples that have been qc'd out
barcodes_across_sample = pd.read_pickle(normalised_barcode_path)

areas = pd.read_csv(lcm_reg_dir/'3d_areas.csv')
areas = areas.merge(RTtosample, how='inner', on='sample')
areas =areas[areas['RT_primer'].isin(barcodes_across_sample.columns)]

regions_to_add = {}
regions_to_add['MG'] = [55, 66]
regions_to_add['LP'] = [30, 39, 54]
regions_to_add['POL'] = [44, 56, 67]
regions_to_add['Rstr'] = [8, 13, 19]
regions_to_add['CLA/Epc'] = [9, 14, 20, 25]
regions_to_add['Cstr'] = [29]
regions_to_add['SCs'] = [87, 68, 78, 92]
regions_to_add['SCm'] = [69, 79, 93]
regions_to_add['IC'] = [86, 89, 90]
regions_to_add['pons'] = [72, 80, 88]
for k, v in regions_to_add.items():
    if k not in areas.columns:
        areas[k] = 0
    for value in v:
        area_of_interest = areas.loc[areas['sample'] == value]
        new_val = area_of_interest.sum(axis=1)-(area_of_interest['RT_primer']+area_of_interest['sample'])
        to_add = pd.DataFrame(columns=areas.columns)
        to_add[k] = new_val
        to_add['sample']= value
        to_add['RT_primer'] = area_of_interest['RT_primer']
        to_add =to_add.fillna(0)
        areas = areas.drop(area_of_interest.index)
        areas= pd.concat([areas, to_add])
areas =areas.set_index('RT_primer').sort_index()

In [ ]:
for group, columns in group_areas.items():
    areas[group] = areas.filter(items=columns).sum(axis=1)
    areas = areas.drop(columns, axis=1)
areas_only = areas.drop(['sample', 'ar', 'bic', 'bsc', 'ccb', 'ccb', 'ccg', 'cing', 'cpd', 'csc', 'cst', 'ec', 'fa', 'fi',
    'fiber tracts', 'fp', 'll', 'mcp', 'ml', 'onl', 'or', 'py', 'root', 'sctv', 'scwm', 'tb', 'CTXsp', 'act', 'alv', 'amc', 'cic', 'VL', 'MRN'], axis=1)


In [ ]:
areas_only = areas_only.loc[:, np.sum(areas_only, axis=0)>0]

Look at barcodes across samples by projection density

In [ ]:
#create a dataframe of the fractions of each brain area contained within each sample
areas_matrix = areas_only.to_numpy()
total_frac = np.sum(areas_matrix, axis=1)
frac_matrix = areas_matrix/total_frac[:, np.newaxis]
areasFrac = pd.DataFrame(frac_matrix, columns=areas_only.columns)

In [ ]:
#for each barcode, create a matrix of BC count for regions in a sample based on amount of each region in LCM (makes assumption of equal BC distribution)
bc_matrix = np.zeros(shape=((len(barcodes_across_sample), (len(areas_only.columns)))))
bc_matrix = pd.DataFrame(data= bc_matrix, columns=areas_only.columns, index=barcodes_across_sample.index)

In [ ]:
for i, row in barcodes_across_sample.iterrows():
    bc_matrix1 = pd.DataFrame(columns=areas_only.columns)
    counts = row.to_numpy()
    frac_counts = frac_matrix * counts[:, np.newaxis]
    sample_counts = pd.DataFrame(frac_counts, columns=areas_only.columns)
    bc_matrix.loc[i] = sample_counts.sum() / areas_only.sum()


In [ ]:
#remove any columns that are all zeros
for column in bc_matrix.columns:
    if bc_matrix[column].sum() == 0:
        bc_matrix.drop([column], axis=1, inplace=True)
bc_matrix = bc_matrix.loc[~(bc_matrix==0).all(axis=1)]

In [ ]:
bc_matrix = bc_matrix.reset_index(drop=True)

In [ ]:
# perform hierarchial clustering of all barcodes across samples
sb.clustermap(bc_matrix, metric='euclidean', standard_scale=0, norm=LogNorm(), yticklabels=False, cmap="PuBuGn", figsize=(20, 10), cbar_kws={'label': 'projection density', "shrink": 0.4}, cbar_pos=(1.01, 0.5, 0.02, 0.18))

Grouping LCM sections according to broad areas

In [ ]:
binary = bc_matrix.astype(bool)*1
sb.clustermap(binary, metric='euclidean', standard_scale=0, norm=LogNorm(), cmap="Blues", figsize=(10, 10))

Since subcortical regions are not great quality, and cortical regions are most of interest, see if any patterns in cortical projections

In [ ]:
areas = pd.read_csv(lcm_reg_dir/'3d_areas.csv')
areas = areas.merge(RTtosample, how='inner', on='sample')
areas.sort_values("RT_primer", inplace=True)
areas =areas[areas['RT_primer'].isin(barcodes_across_sample.columns)]
cortical_group_areas = {
    'SSp': ['SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-n', 'SSp-tr', 'SSp-ul', 'SSp-un'],
    'contra': areas.filter(like="Contra").columns,
}
for group, columns in cortical_group_areas.items():
    areas[group] = areas.filter(items=columns).sum(axis=1)
    areas = areas.drop(columns, axis=1)
areas = areas.drop(['sample', 'ar', 'bic', 'bsc', 'ccb', 'ccb', 'ccg', 'cing', 'cpd', 'csc', 'cst', 'ec', 'fa', 'fi',
    'fiber tracts', 'fp', 'll', 'mcp', 'ml', 'onl', 'or', 'py', 'root', 'sctv', 'scwm', 'tb', 'CTXsp', 'act', 'alv', 'amc', 'cic', 'VL', 'MRN'], axis=1)
areas =areas.set_index('RT_primer')

In [ ]:
#create a dataframe of the fractions of each brain area contained within each sample
areas_matrix = areas.to_numpy()
total_frac = np.sum(areas_matrix, axis=1)
frac_matrix = areas_matrix/total_frac[:, np.newaxis]
areasFrac = pd.DataFrame(frac_matrix, columns=areas.columns)

#for each barcode, create a matrix of BC count for regions in a sample based on amount of each region in LCM (makes assumption of equal BC distribution)
bc_matrix = np.zeros(shape=((len(barcodes_across_sample), (len(areas.columns)))))
bc_matrix = pd.DataFrame(data= bc_matrix, columns=areas.columns, index=barcodes_across_sample.index)
for i, row in barcodes_across_sample.iterrows():
    counts = row.to_numpy()
    frac_counts = frac_matrix * counts[:, np.newaxis]
    sample_counts = pd.DataFrame(frac_counts, columns=areas.columns)
    bc_matrix.loc[i] = sample_counts.sum() / areas.sum()



In [ ]:
cortex = ['ACAd', 'ACAv', 'ECT', 'MOp', 'MOs', 'RSPagl', 'RSPd', 'RSPv', 'SSp', 'SSs', 'VISC', 'VISal',
'VISam', 'VISl', 'VISli', 'VISp', 'VISpl','VISpm', 'VISpor', 'VISrl', 'contra', 'TEa', 'AUDp', 'AUDv', 'AUDd']
bc_matrix = bc_matrix[cortex]
#remove any columns that are all zeros
for column in bc_matrix.columns:
    if bc_matrix[column].sum() == 0:
        bc_matrix.drop([column], axis=1, inplace=True)
bc_matrix = bc_matrix.loc[~(bc_matrix==0).all(axis=1)]


In [ ]:
bc_matrix = bc_matrix.reset_index(drop=True)

In [ ]:
sb.clustermap(bc_matrix, metric='canberra', standard_scale=0, norm=LogNorm(), yticklabels=False, cmap="PuBuGn", figsize=(10, 10), cbar_kws={'label': 'projection density', "shrink": 0.4}, cbar_pos=(1.01, 0.5, 0.02, 0.18))

In [ ]:
areasFrac[(areasFrac['TEa']>0) & (areasFrac['VISpm']>0)]

In [ ]:
#cortical projections weighted by relationship between area size and counts
#areas_only = areas.loc[:, np.sum(areas, axis=0)>0]
areas_only = areas_only.loc[:, np.sum(areas_only, axis=0)>0]
areas_matrix = areas_only.to_numpy()
areas_matrix /= np.sum(areas_matrix, axis=0)
barcodes_across_sample.fillna(0,inplace=True)
barcodes_matrix = barcodes_across_sample.to_numpy()
barcodes_matrix[np.isnan(barcodes_matrix)] = 0
total_projection_strength = np.sum(barcodes_matrix, axis=1)
barcodes_matrix /= total_projection_strength[:, np.newaxis]
barcodes_matrix = barcodes_matrix[total_projection_strength>0, :]
from sklearn.linear_model import LinearRegression, Lasso

mdl = LinearRegression(fit_intercept=False, positive=True)
mdl.fit(areas_matrix, barcodes_matrix.T)

In [ ]:
df = pd.DataFrame(mdl.coef_, columns=areas_only.columns)
cortex = ['ACAd', 'ACAv', 'ECT', 'MOp', 'MOs', 'RSPagl', 'RSPd', 'RSPv', 'SSp', 'SSs', 'VISal',
'VISam', 'VISl', 'VISli', 'VISp', 'VISpl','VISpm', 'VISpor', 'VISrl', 'contra', 'TEa', 'AUDp', 'AUDv', 'AUDd']
cortex_df = df[cortex]
#cortex_df =cortex_df[cortex_df.sum(axis=1)>0.001]

In [ ]:
plt.hist(df.melt()['value'], bins=30)
plt.yscale('log')

In [ ]:

sb.clustermap(df.T, metric='euclidean', vmax=0.05, dendrogram_ratio=[0.1, 0.1], yticklabels=True, cmap='Blues', figsize=(10, 10))
#sb.clustermap(cortex_df.T, metric='euclidean', standard_scale=0, norm=LogNorm(), cmap="Blues", figsize=(10, 10))

In [ ]:
sb.clustermap(cortex_df.T, metric='euclidean', vmax=0.05, cmap='Blues', yticklabels=True, figsize=(10, 10))

In [ ]:
barcodes_across_sample[36]


In [ ]:
areas[(areas['AUDv']>0) & (areas['AUDp']>0)]

In [ ]:
areas_only[areas_only['contra']>0]

In [ ]:
areas[areas['AUDp']>0]

In [ ]:
#create a dataframe of the fractions of each brain area contained within each sample
cortex_areas_matrix = cortical_areas.to_numpy()
cortex_total_frac = np.sum(cortex_areas_matrix, axis=1)
cortex_frac_matrix = cortex_areas_matrix/cortex_total_frac[:, np.newaxis]
cortex_areasFrac = pd.DataFrame(cortex_frac_matrix, columns=cortical_areas.columns)

In [ ]:
for i, row in barcodes_across_sample.iterrows():
    counts = row.to_numpy()
    frac_counts = cortex_frac_matrix * counts[:, np.newaxis]
    sample_counts = pd.DataFrame(frac_counts, columns=cortical_areas.columns)
    cortex_bc_matrix.loc[i] = sample_counts.sum() / cortical_areas.sum()

In [ ]:
#for each barcode, create a matrix of BC count for regions in a sample based on amount of each region in LCM (makes assumption of equal BC distribution)
bc_matrix = np.zeros(shape=((len(barcodes_across_sample), (len(areas_only_grouped.columns)))))
bc_matrix = pd.DataFrame(data= bc_matrix, columns=areas_only_grouped.columns, index=barcodes_across_sample.index)
for i, row in barcodes_across_sample.iterrows():
    counts = row.to_numpy()
    frac_counts = frac_matrix * counts[:, np.newaxis]
    sample_counts = pd.DataFrame(frac_counts, columns=areas_only_grouped.columns)
    bc_matrix.loc[i] = sample_counts.sum() / areas_only_grouped.sum()
#remove any columns that are all zeros
for column in bc_matrix.columns:
    if bc_matrix[column].sum() == 0:
        bc_matrix.drop([column], axis=1, inplace=True)
bc_matrix = bc_matrix.loc[~(bc_matrix==0).all(axis=1)]

In [ ]:
cortex = ['ACAd', 'ACAv', 'AUDd', 'AUDp', 'AUDpo', 'AUDv', 'ECT', 'MOp', 'MOs', 'RSPagl', 'RSPd', 'RSPv', 'SSp', 'SSs', 'TEa', 'VISC', 'VISal',
'VISam', 'VISl', 'VISli', 'VISp', 'VISpl','VISpm', 'VISpor', 'VISrl', 'contra']
cortical_areas = cortical_areas[cortex]
#create a dataframe of the fractions of each brain area contained within each sample
cortical_areas_matrix = cortical_areas.to_numpy()
total_frac_cortex = np.sum(cortical_areas_matrix, axis=1)
cortical_frac_matrix = cortical_areas_matrix/total_frac_cortex[:, np.newaxis]
cortical_areasFrac = pd.DataFrame(cortical_frac_matrix, columns=cortical_areas.columns)



Are there any patterns between visual areas?

In [ ]:
areas_comp = ['VISa', 'VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISpor', 'VISrl', 'VISli', 'VISpl', 'SSs', 'SSp', 'MOp', 'MOs', 'ENTl', 'ECT', 'AUDpo']
vis_matrix = bc_matrix[areas_comp]

In [ ]:
vis_matrix = vis_matrix.loc[~(vis_matrix==0).all(axis=1)]

In [ ]:
sb.clustermap(vis_matrix, metric='euclidean', standard_scale=0, norm=LogNorm(), cmap="Blues", figsize=(20, 10))

In [ ]:
areas[['VISpor','ENTl']]

In [ ]:
thalamus_and_tect_comp = ['POL', 'LP', 'SCs', 'SCm', 'IC']
thalamus_and_tect_matrix = bc_matrix[thalamus_and_tect_comp]
thalamus_and_tect_matrix = thalamus_and_tect_matrix.loc[~(thalamus_and_tect_matrix==0).all(axis=1)]
sb.clustermap(thalamus_and_tect_matrix, metric='euclidean', standard_scale=0, norm=LogNorm(), cmap="Blues", figsize=(20, 10))

In [ ]:
sb.clustermap(vis_matrix,